In [54]:
import pandas as pd

lappis_data = pd.read_csv('../data/github_data_lappis.csv')
lappis_data_gitlab = pd.read_csv('../data/gitlab_data_lappis.csv')

lappis_github_selected = lappis_data[['org', 'repo', 'user', 'commits', 'commit_history', 'pull_requests', 'issues', 'languages']]

lappis_gitlab_selected = lappis_data_gitlab[['group', 'project', 'name', 'commits', 'commit_history', 'merge_requests', 'issues', 'languages']]

lappis_gitlab_selected.rename(columns={
    'group': 'org',
    'project': 'repo',
    'name': 'user',
    'merge_requests': 'pull_requests'
}, inplace=True)

lappis_combined = pd.concat([lappis_github_selected, lappis_gitlab_selected])

lappis_combined.to_csv('../data/lappis_combined.csv', index=False)

/tmp/ipykernel_914850/3857445168.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lappis_gitlab_selected.rename(columns={


In [55]:
import pandas as pd

lappis_data = pd.read_csv('../data/lappis_combined.csv')
eps_data = pd.read_csv('../data/github_data_eps_mds.csv')

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

lappis_data_filtered = remove_outliers(lappis_data, 'commits')
eps_data_filtered = remove_outliers(eps_data, 'commits')


In [56]:
repos_lappis = lappis_data_filtered['repo'].nunique()
repos_eps = eps_data_filtered['repo'].nunique()

sample_size = min(repos_lappis, repos_eps)

def amostrar_repositorios(df, sample_size):
    repositorios_amostrados = df['repo'].drop_duplicates().sample(n=sample_size, random_state=42)
    df_amostrado = df[df['repo'].isin(repositorios_amostrados)]
    return df_amostrado

balanced_lappis = amostrar_repositorios(lappis_data, sample_size)
balanced_eps = amostrar_repositorios(eps_data, sample_size)

In [57]:
import pandas as pd

def calculate_repository_summary(df):
    total_repositories = df['repo'].nunique()
    
    average_commits = df.groupby('repo')['commits'].sum().mean()
    
    average_merge_requests = df.groupby('repo')['pull_requests'].sum().mean()
    
    average_issues = df.groupby('repo')['issues'].sum().mean()
    
    return {
        'Total Repositories': total_repositories,
        'Average Commits per Repository': average_commits,
        'Average Merge Requests per Repository': average_merge_requests,
        'Average Issues per Repository': average_issues
    }

metrics_lappis = calculate_repository_summary(balanced_lappis)
metrics_eps = calculate_repository_summary(balanced_eps)

summary_lappis = pd.DataFrame([metrics_lappis], index=['Lappis'])
summary_eps = pd.DataFrame([metrics_eps], index=['EPS'])

summary_combined = pd.concat([summary_lappis, summary_eps])

print(summary_combined)


        Total Repositories  Average Commits per Repository  \
Lappis                 147                      281.360544   
EPS                    147                      297.149660   

        Average Merge Requests per Repository  Average Issues per Repository  
Lappis                              10.367347                      22.755102  
EPS                                 33.632653                      76.408163  
